In [1]:
# 데이터셋을 DataLoader로 변환하고, 모델을 정의하며, 학습 및 평가 루프를 직접 구현하는 과정

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

# 파이토치 기본 공식 베이직 코드를 참조함
# ds = datasets.FashionMNIST(
#     root="data",
#     train=True,
#     download=True,
#     transform=ToTensor(),
#     target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
# )

# 원-핫 인코딩을 하는 과정
# '셔츠'를 나타내는 레이블 y = 6이 이 함수에 전달될 때 가정,
# torch.zeros(10, ...)이 [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.] 텐서를 생성.
# .scatter_ 메소드가 이 텐서를 받습니다. -> _의미: 제자리에서 작업이 이루어진다는 뜻
# torch.tensor(y)가 torch.tensor(6)이 되므로, 인덱스 6을 사용.
# 인덱스 6의 위치에 value=1을 넣습니다.
# 그 결과, [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]와 같은 텐서가 완성

In [24]:
# 데이터셋 준비
training_data = datasets.FashionMNIST(
  root="data",
  train=True,
  download=True,
  transform=ToTensor(),
  target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

test_data = datasets.FashionMNIST(
  root="data",
  train=False,
  download=True,
  transform=ToTensor(),
  target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

In [18]:
# DataLoader 생성
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [25]:
# Vision Transformer (VIT)모델 정의
# ViT는 이미지를 패치로 분할한 후, 각 패치를 시퀀스로 다룹니다.
# cnn은 필터를 이동시켜 이미지의 지역적 특징을 추출하는 반면에 ViT는 이미지들을 작은 크기의 여러 패치로 나눈 후, 각 패치를 마치 문장의 단어처럼 다룹니다.
# 이 패치들의 순서 시퀀스를 트랜스포머 모델에 입력하여 전체 이미지의 특징을 학습합니다.

class SimpleViT(nn.Module):
  def __init__(self, in_channels=1, patch_size=4, embed_dim=128, num_layers=2, num_heads=4, num_classes=10):
    super().__init__()

    image_size = 28

    # 1. 패치 임베딩: 이미지를 패치로 나누고 각 패치를 벡터로 변환
    # 28x28 이미지를 4x4 크기의 겹치지 않는(stride=4) 조각들로 나눕니다. 이렇게 만들어진 각 패치는 embed_dim (128) 차원의 벡터로 변환.
    self.patch_embed = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)

    # 2. 위치 인코딩: 패치들의 순서 정보를 추가
    # num_patches + 1은 패치 개수(28/4 * 28/4 = 49)에 특별한 [CLS] 토큰 한 개를 더한 것입니다.
    num_patches = (image_size // patch_size) * (image_size // patch_size)
    self.positions = nn.Parameter(torch.randn(num_patches + 1, embed_dim))

    # 3.[CLS] 토큰: 시퀀스 전체를 대표하는 특별한 토큰
    # 트랜스포머 인코더는 이 토큰을 통해 전체 패치들의 정보를 한곳에 모읍니다. 학습이 끝나면 이 [CLS] 토큰의 출력이 전체 이미지를 대표하는 벡터가 됩니다. 우리는 이 벡터를 분류에 사용
    self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))

    # 4. 트랜스포머 인코더: 패치 간의 관계를 학습
    # nhead=num_heads 어텐션 헤드의 수 -> 어텐션 메커니즘이 한 번에 여러 관점에서 패치 간의 관계를 탐색
    encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, batch_first=True)
    self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

    # 5. 분류 헤드: 최종 예측을 수행합니다.
    self.classifier = nn.Linear(embed_dim, num_classes)

  def forward(self, x):
    # 이미지를 패치로 변환 (N, C, H, W) -> (N, P, E)
    # flatten(2): 패치들을 1차원 시퀀스로 펼칩니다.
    # transpose(1, 2):차원의 순서를 바꿉니다. 트랜스포머 입력에 맞는 (배치_크기, 패치_개수, 임베딩_차원) 형태로 만듭니다.
    x = self.patch_embed(x)
    x = x.flatten(2).transpose(1, 2)

    # [CLS] 토큰을 패치 시퀀스에 추가
    cls_tokens = self.cls_token.expand(x.shape[0], -1, -1) # 0은 미니배치 데이터, -1은 해당 차원의 크기를 그대로
    x = torch.cat((cls_tokens, x), dim=1)

    # 위치 임베딩 추가
    x += self.positions

    # 트랜스포머 인코더에 통과
    x = self.transformer_encoder(x)

    # [CLS] 토큰의 출력을 사용하여 분류
    cls_token_output = x[:, 0]
    logits = self.classifier(cls_token_output)
    return logits

# 토큰은 시퀀스의 가장 앞에 배치되지만, 그 자체로는 어떤 정보도 가지고 있지 않습니다. 이 토큰의 역할은 오직 다른 모든 패치(이미지 조각)들로부터 정보를 모으고 요약하는 것입니다.
# 트랜스포머의 핵심인 어텐션(Self-Attention) 메커니즘을 통해 이 작업이 이루어집니다.

# 모델 인스턴스 생성
model = SimpleViT()

# 이미지 -> (패치로 분할) -> [CLS] 토큰과 위치 정보가 추가된 패치 시퀀스 -> 트랜스포머 인코더에 통과 -> [CLS] 토큰의 출력을 사용 -> 분류기에서 최종 예측.

In [26]:
# 손실 함수와 최적화기 정의 (이전과 동일)
# 손실 함수(Loss Function): 모델의 예측과 실제 레이블 간의 차이를 계산합니다.
# 최적화기(Optimizer): 손실 함수 값을 줄이기 위해 모델의 파라미터(가중치)를 업데이트합니다.
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# 학습 루프 정의
# 이 함수는 DataLoader를 순회하며 모델을 학습시킵니다.
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() # 모델을 학습 모드로 설정
    for batch, (X, y) in enumerate(dataloader):
        # 예측 및 손실 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파(Backpropagation)
        optimizer.zero_grad() # 이전 기울기 초기화
        loss.backward()       # 손실에 대한 기울기 계산
        optimizer.step()      # 가중치 업데이트

        # 학습 진행 상황 출력
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

# 테스트 루프 정의
# 학습된 모델의 성능을 평가합니다.
def test_loop(dataloader, model, loss_fn):
    model.eval() # 모델을 평가 모드로 설정
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad(): # 기울기 계산을 비활성화하여 메모리를 절약하고 속도를 높입니다.
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            # 이 한 줄은 "모델이 예측한 클래스 인덱스"와 "실제 정답의 클래스 인덱스"를 비교하여, 예측이 일치하는 샘플의 개수를 세는 코드입니다.
            # 이 개수를 전체 샘플 수로 나누면 정확도(Accuracy)를 계산할 수 있습니다.
            correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# 학습 실행
# 지정된 epoch 수만큼 학습과 평가를 반복합니다.
epochs = 2
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 2.505097  [   64/60000]
loss: 1.009578  [ 6464/60000]
loss: 0.955084  [12864/60000]
loss: 0.670252  [19264/60000]
loss: 0.761983  [25664/60000]
loss: 0.715345  [32064/60000]
loss: 0.652899  [38464/60000]
loss: 0.705130  [44864/60000]
loss: 0.537867  [51264/60000]
loss: 0.563349  [57664/60000]
Test Error: 
 Accuracy: 79.4%, Avg loss: 0.559000 

Epoch 2
-------------------------------
loss: 0.624593  [   64/60000]
loss: 0.747637  [ 6464/60000]
loss: 0.731733  [12864/60000]
loss: 0.461139  [19264/60000]
loss: 0.630891  [25664/60000]
loss: 0.727792  [32064/60000]
loss: 0.619790  [38464/60000]
loss: 0.596877  [44864/60000]
loss: 0.361807  [51264/60000]
loss: 0.594487  [57664/60000]
Test Error: 
 Accuracy: 81.3%, Avg loss: 0.517823 

Done!
